In [1]:
import lightgbm as lgb

In [2]:
import pandas as pd
import numpy as np
from category_encoders.target_encoder import TargetEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import auc, roc_curve
from lightgbm import LGBMRegressor

ModuleNotFoundError: No module named 'category_encoders'

In [ ]:
# 导入数据
train = pd.read_csv('train.csv', index_col='id')
test = pd.read_csv('test.csv', index_col='id')
target = train.pop('fraud_flag')
test = test[train.columns]

In [ ]:
train

,base_month_income,base_car_km,base_loan_rate,base_loan_monthrepay,base_car_years,base_ages,br_scoreafautofin,ir_id_x_cell_cnt,ir_m12_id_x_home_addr_cnt,ir_allmatch_days,...,base_has_rebuy,base_sex,zx_result,zx_is_baihu,zx_is_current_ovd,zx_is_credictcard_current_ovd,zx_is_lian3_lei6,loan_is_black,base_has_driverlicensce,zx_account_status
id,,,,,,,,,,,,,,,,,,,,,
Jm1vdsMO/XNuR3ES3XN9bPn9LpUeHz10ytfCjbyFfZA=,2,6,3,5,5,1,5,2,-1,7,...,-1,1,-1,-1,-1,-1,-1,-1,2,-1
+0RRLCEvWPup/NJMBgQbnArYOZuUCqXE+3wvlISLJ3M=,3,3,3,4,4,4,-1,-1,-1,-1,...,-1,-1,1,-1,-1,-1,-1,-1,1,-1
waiwhkaUgGnbMsiboLPW9zv/V5UdKq5QqfKb5458DwA=,2,2,3,1,2,1,-1,2,-1,-1,...,-1,-1,1,1,-1,-1,-1,-1,2,-1
+8AhpQt6enebe5Ce+P4qc1MnPbuguH/vGszadhblZIo=,2,2,3,3,2,2,6,-1,-1,-1,...,-1,1,1,-1,-1,-1,-1,-1,2,-1
1S+aUkQe8plPsGR/WCStlboXYW5shXOnEv4qM8yzlw4=,2,5,1,1,5,4,5,3,3,6,...,-1,1,1,-1,-1,-1,-1,-1,2,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3o6T5lTFjv1wX/gV1s4uim/Rh47ATNWAD/EcAZrSs2c=,3,2,3,3,4,2,6,4,3,2,...,-1,1,1,-1,-1,-1,-1,-1,2,-1
73y0W/Vb6d2TnMy8e85ecE7Jxn1aTNJlAwTEwXGRaPs=,1,6,3,5,5,5,-1,-1,-1,-1,...,1,1,1,1,-1,-1,-1,-1,3,-1
QLK+7cm2wkT7gnjWCaLtjz21QEH64543A9U/2/b9x6A=,4,4,2,3,5,2,5,2,4,2,...,-1,1,1,-1,-1,-1,-1,1,1,-1


In [ ]:
# 非数值列
s = train.apply(lambda x:x.dtype)
tecols = s[s=='object'].index.tolist()

In [ ]:
# 模型
def makelgb():
    lgbr = LGBMRegressor(num_leaves=30
                        ,max_depth=5
                        ,learning_rate=.02
                        ,n_estimators=1000
                        ,subsample_for_bin=5000
                        ,min_child_samples=200
                        ,colsample_bytree=.2
                        ,reg_alpha=.1
                        ,reg_lambda=.1
                        )
    return lgbr

# 本地验证
kf = KFold(n_splits=10, shuffle=True, random_state=100)
devscore = []
for tidx, didx in kf.split(train.index):
    tf = train.iloc[tidx]
    df = train.iloc[didx]
    tt = target.iloc[tidx]
    dt = target.iloc[didx]
    te = TargetEncoder(cols=tecols)
    tf = te.fit_transform(tf, tt)
    df = te.transform(df)
    lgbr = makelgb()
    lgbr.fit(tf, tt)
    pre = lgbr.predict(df)
    #print(type(dt), type(pre))
    threshold_rate  = 0.68
    pre= np.where(pre > np.quantile(pre, threshold_rate), 1, 0)
    fpr, tpr, thresholds = roc_curve(dt, pre)
    score = auc(fpr, tpr)
    print(score)
    devscore.append(score)
print("Average Score")
print(np.mean(devscore))

0.713494623655914
0.6725774225774224
0.7067135835567472
0.6193939393939394
0.7988709677419354
0.7067135835567472
0.7604627766599598
0.7366473013493253
0.7063172043010754
0.680447242318895
Average Score
0.7101638645111961


In [ ]:
test

,base_month_income,base_car_km,base_loan_rate,base_loan_monthrepay,base_car_years,base_ages,br_scoreafautofin,ir_id_x_cell_cnt,ir_m12_id_x_home_addr_cnt,ir_allmatch_days,...,base_has_rebuy,base_sex,zx_result,zx_is_baihu,zx_is_current_ovd,zx_is_credictcard_current_ovd,zx_is_lian3_lei6,loan_is_black,base_has_driverlicensce,zx_account_status
id,,,,,,,,,,,,,,,,,,,,,
SbTOgz2IEr5FULK7toY5hVLMPbHuKJxDQQr4L2GIklI=,2,2,2,3,4,1,5,2,2,2,...,-1,-1,1,-1,-1,-1,-1,-1,2,-1
pulzuTR48W6C4vX12F/NBr9XfEhal47N73MSOMP881M=,3,3,3,4,4,2,5,2,-1,-1,...,-1,1,1,-1,-1,-1,-1,-1,2,-1
+atpROxhINIH1xGIOpoG3VMnPbuguH/vGszadhblZIo=,2,4,2,1,5,1,4,2,3,5,...,-1,1,1,-1,-1,-1,-1,-1,2,-1
+BPaVduqYh8FhYsSWURxdnfsV4irW0L2ZNtxBFAEeYc=,2,4,3,2,5,3,5,2,3,3,...,-1,1,1,-1,-1,-1,-1,-1,2,-1
+c6ODxtEJiaK0ebmsA9a8oKk+RgpxZ5CquL6pjPLrHo=,4,2,1,5,2,3,4,2,-1,7,...,-1,-1,1,-1,-1,-1,-1,-1,2,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DdlzJLMjN5TMKg5H1FKZd2jZgyvJwnnfXsIcpioBhXE=,3,4,1,5,5,2,-1,3,4,3,...,-1,1,1,-1,-1,-1,-1,-1,2,-1
kuqemz2WC0NOVQBosIAQcbCDC0U18L/PGRAM+YpMFWU=,4,3,1,4,4,3,-1,4,2,6,...,-1,1,1,-1,-1,-1,-1,-1,2,-1
IiywSJod13VIzLxQPNisDL0ZvuRySPubS9lBbijN3Rw=,1,4,3,4,5,1,3,2,2,6,...,-1,1,1,1,-1,-1,-1,-1,2,-1


In [ ]:
# 在整个train集上重新训练，预测test，输出结果
lgbr = makelgb()
te = TargetEncoder(cols=tecols)
tf = te.fit_transform(train, target)
df = te.transform(test)
lgbr.fit(tf, target)
pre = lgbr.predict(te.transform(train))
fpr, tpr, thresholds = roc_curve(target, pre)
score = auc(fpr, tpr)
print(score)

for i in range(65, 95, 1):
    threshold_rate  = i/100
    fraud_list = np.where(pre > threshold_rate, 0, 1)
    print('阈值选择 {} 时, 预测有 {} 个fraud用户:'.format(threshold_rate, sum(np.where(pre > np.quantile(pre, threshold_rate), 1, 0))))
threshold_rate  = 0.68
pre= np.where(pre > np.quantile(pre, threshold_rate), 1, 0)
print(sum(pre))
pd.Series(pre, name='label', index=test.index).reset_index().to_csv('FinalSubmit.csv', index=False)


0.9154715294398416
阈值选择 0.65 时, 预测有 1389 个fraud用户:
阈值选择 0.66 时, 预测有 1349 个fraud用户:
阈值选择 0.67 时, 预测有 1310 个fraud用户:
阈值选择 0.68 时, 预测有 1270 个fraud用户:
阈值选择 0.69 时, 预测有 1230 个fraud用户:
阈值选择 0.7 时, 预测有 1191 个fraud用户:
阈值选择 0.71 时, 预测有 1151 个fraud用户:
阈值选择 0.72 时, 预测有 1111 个fraud用户:
阈值选择 0.73 时, 预测有 1072 个fraud用户:
阈值选择 0.74 时, 预测有 1032 个fraud用户:
阈值选择 0.75 时, 预测有 992 个fraud用户:
阈值选择 0.76 时, 预测有 953 个fraud用户:
阈值选择 0.77 时, 预测有 913 个fraud用户:
阈值选择 0.78 时, 预测有 873 个fraud用户:
阈值选择 0.79 时, 预测有 834 个fraud用户:
阈值选择 0.8 时, 预测有 794 个fraud用户:
阈值选择 0.81 时, 预测有 754 个fraud用户:
阈值选择 0.82 时, 预测有 715 个fraud用户:
阈值选择 0.83 时, 预测有 675 个fraud用户:
阈值选择 0.84 时, 预测有 635 个fraud用户:
阈值选择 0.85 时, 预测有 596 个fraud用户:
阈值选择 0.86 时, 预测有 556 个fraud用户:
阈值选择 0.87 时, 预测有 516 个fraud用户:
阈值选择 0.88 时, 预测有 477 个fraud用户:
阈值选择 0.89 时, 预测有 437 个fraud用户:
阈值选择 0.9 时, 预测有 397 个fraud用户:
阈值选择 0.91 时, 预测有 358 个fraud用户:
阈值选择 0.92 时, 预测有 318 个fraud用户:
阈值选择 0.93 时, 预测有 278 个fraud用户:
阈值选择 0.94 时, 预测有 239 个fraud用户:
1270


ValueError: Length of values (3968) does not match length of index (2574)

In [ ]:
te.transform(train)

,base_month_income,base_car_km,base_loan_rate,base_loan_monthrepay,base_car_years,base_ages,br_scoreafautofin,ir_id_x_cell_cnt,ir_m12_id_x_home_addr_cnt,ir_allmatch_days,...,base_has_rebuy,base_sex,zx_result,zx_is_baihu,zx_is_current_ovd,zx_is_credictcard_current_ovd,zx_is_lian3_lei6,loan_is_black,base_has_driverlicensce,zx_account_status
id,,,,,,,,,,,,,,,,,,,,,
Jm1vdsMO/XNuR3ES3XN9bPn9LpUeHz10ytfCjbyFfZA=,2,6,3,5,5,1,5,2,-1,7,...,-1,1,-1,-1,-1,-1,-1,-1,2,-1
+0RRLCEvWPup/NJMBgQbnArYOZuUCqXE+3wvlISLJ3M=,3,3,3,4,4,4,-1,-1,-1,-1,...,-1,-1,1,-1,-1,-1,-1,-1,1,-1
waiwhkaUgGnbMsiboLPW9zv/V5UdKq5QqfKb5458DwA=,2,2,3,1,2,1,-1,2,-1,-1,...,-1,-1,1,1,-1,-1,-1,-1,2,-1
+8AhpQt6enebe5Ce+P4qc1MnPbuguH/vGszadhblZIo=,2,2,3,3,2,2,6,-1,-1,-1,...,-1,1,1,-1,-1,-1,-1,-1,2,-1
1S+aUkQe8plPsGR/WCStlboXYW5shXOnEv4qM8yzlw4=,2,5,1,1,5,4,5,3,3,6,...,-1,1,1,-1,-1,-1,-1,-1,2,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3o6T5lTFjv1wX/gV1s4uim/Rh47ATNWAD/EcAZrSs2c=,3,2,3,3,4,2,6,4,3,2,...,-1,1,1,-1,-1,-1,-1,-1,2,-1
73y0W/Vb6d2TnMy8e85ecE7Jxn1aTNJlAwTEwXGRaPs=,1,6,3,5,5,5,-1,-1,-1,-1,...,1,1,1,1,-1,-1,-1,-1,3,-1
QLK+7cm2wkT7gnjWCaLtjz21QEH64543A9U/2/b9x6A=,4,4,2,3,5,2,5,2,4,2,...,-1,1,1,-1,-1,-1,-1,1,1,-1


In [ ]:
te

TargetEncoder(cols=[])